In [1]:
# 你想定义一个生成器函数，它会调用某个你想暴露给用户使用的外部状态值。
# 也就是你想访问生成器中某个参数值，或者某个过程变量！

In [2]:
# 如果你想让你的生成器暴露外部状态给用户， 别忘了你可以简单的将它实现为一个类，然后把生成器函数放到 __iter__() 方法中过去。比如：

from collections import deque


class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)

    def __iter__(self):
        # enumerate 第二参数为第一个index序号为1，默认为0
        for line_no, line in enumerate(self.lines, 1):
            self.history.append((line_no, line))
            yield line

    def clear(self):
        self.history.clear()


为了使用这个类，你可以将它当做是一个普通的生成器函数。 然而，由于可以创建一个实例对象，于是你可以访问内部属性值， 比如 history 属性或者是 clear() 方法。代码示例如下：

In [5]:
with open('/etc/pam.d/passwd') as f:
    lines = linehistory(f)
    for line in lines:
        if 'auth' in line:
            for line_no, hline in lines.history:
                print('{}:{}'.format(line_no, hline), end='')


1:# passwd: auth account
1:# passwd: auth account
2:auth       required       pam_permit.so


关于生成器，很容易掉进函数无所不能的陷阱。 如果生成器函数需要跟你的程序其他部分打交道的话(比如暴露属性值，允许通过方法调用来控制等等)， 可能会导致你的代码异常的复杂。 如果是这种情况的话，可以考虑使用上面介绍的定义类的方式。 在 __iter__() 方法中定义你的生成器不会改变你任何的算法逻辑。 由于它是类的一部分，所以允许你定义各种属性和方法来供用户使用。



一个需要注意的小地方是，如果你在迭代操作时不使用for循环语句，那么你得先调用 iter() 函数。比如：

In [14]:
f = open('/etc/pam.d/passwd')
lines = linehistory(f)
print(lines)
next(lines)

TypeError: 'linehistory' object is not an iterator

In [8]:
it = iter(lines)

In [9]:
it

<generator object linehistory.__iter__ at 0x103a2b780>

In [10]:
next(it)

'# passwd: auth account\n'

In [11]:
next(it)

'auth       required       pam_permit.so\n'